In [1]:
# Imports
%load_ext autoreload
%autoreload 2

import json
from dwitracts.main import DwiTracts
import dwitracts.plot as plot

In [49]:
# Initialize a DwiTract project
config_file = 'project_full/config_tracts_dmn.json'

def _json_object_hook(d): return namedtuple('X', d.keys())(*d.values())
def json2obj(data): return json.loads(data, object_hook=_json_object_hook)

with open(config_file, 'r') as myfile:
    json_string=myfile.read()

params = json.loads(json_string)

my_dwi = DwiTracts( params )

assert my_dwi.initialize()


Creating debug directory at /Volumes/AndrewElements/data/nki/tract_stats/dmn/debug


In [ ]:
# Compute bidirectional average distributions for each tract ([AB + BA] / 2)

assert my_dwi.compute_bidirectional_averages( verbose=True, clobber=True )


In [ ]:
# Compute tract distances (A->B and B->A) for each tract

assert my_dwi.compute_tract_distances( verbose=True, clobber=True )


In [ ]:
# Generate core polylines, gaussian uncertainty fields, and unidirectional tract estimates 

assert my_dwi.estimate_unidirectional_tracts( verbose=True, clobber=True )


In [56]:
# Generate bidirectional tract estimates 

assert my_dwi.estimate_bidirectional_tracts( verbose=True, clobber=True )



== Computing bidirectional tracts...


  Wrote average bidirectional tract for ACC_L/LOC_L
  Wrote average bidirectional tract for ACC_L/LOC_R
  Wrote average bidirectional tract for ACC_L/PCG_L
  Wrote average bidirectional tract for ACC_L/PCG_R
  Wrote average bidirectional tract for ACC_L/PCm_L
  Wrote average bidirectional tract for ACC_L/PCm_R
  Wrote average bidirectional tract for ACC_L/PFCm_L
  Wrote average bidirectional tract for ACC_L/PFCm_R
  Wrote average bidirectional tract for LOC_L/LOC_R
  Wrote average bidirectional tract for LOC_L/PCG_L
  Wrote average bidirectional tract for LOC_L/PCG_R
  * Tract had multiple tract segments (1 retained): LOC_L|PCm_L
 Tract LOC_L_PCm_L failed.
  Wrote average bidirectional tract for LOC_L/PCm_R
  Wrote average bidirectional tract for LOC_L/PFCm_L
  Wrote average bidirectional tract for LOC_L/PFCm_R
  Wrote average bidirectional tract for LOC_R/PCG_L
  Wrote average bidirectional tract for LOC_R/PCG_R
  * Tract had multiple tract segments (1 retained): LOC_R|PCm_L
 Tract LO

In [ ]:
# Create a pass/fail Pajek graph

assert my_dwi.tracts_to_pajek( verbose=True, clobber=True )


In [ ]:
# Compute voxel-wise average streamline orientations for each tract

assert my_dwi.compute_average_orientations( verbose=True, clobber=True )


In [ ]:
# Compute tract-specific anisotropy

assert my_dwi.compute_tsa( verbose=True, clobber=True )


In [50]:
# Plot TSA histograms
params_plot = {}
params_plot['axis_font'] = 18
params_plot['ticklabel_font'] = 12
params_plot['title_font'] = 18
params_plot['show_labels'] = False
params_plot['show_title'] = False
params_plot['dimensions_tracts'] = (50,40)
params_plot['dimensions_all'] = (50,40)
params_plot['dpi_tracts'] = 150
params_plot['dpi_all'] = 300
params_plot['num_bins'] = 20
params_plot['kde'] = True
params_plot['stat'] = 'density'
params_plot['xlim'] = [-0.1, 0.3]
params_plot['xticks'] = [0.0, 0.1, 0.2, 0.3]
# params_plot['color'] = '#06b70b'  # green [WWN]
params_plot['color'] = '#2b3ad1'  # blue [DMN]

tract_names = None

stats = plot.plot_tsa_histograms( params_plot, my_dwi, tract_names=tract_names, threshold=0.5, \
                                  verbose=True, clobber=True )
stats.to_csv('{0}/stats_tsa.csv'.format(my_dwi.tracts_dir))


 Done tract ACC_L_LOC_L
 Done tract ACC_L_LOC_R
 Done tract ACC_L_PCG_L
 Done tract ACC_L_PCG_R
 Done tract ACC_L_PCm_L
 Done tract ACC_L_PCm_R
 Done tract ACC_L_PFCm_L
 Done tract ACC_L_PFCm_R
 Done tract LOC_L_LOC_R
 Done tract LOC_L_PCG_L
 Done tract LOC_L_PCG_R
 Done tract LOC_L_PCm_R
 Done tract LOC_L_PFCm_L
 Done tract LOC_L_PFCm_R
 Done tract LOC_R_PCG_L
 Done tract LOC_R_PCG_R
 Done tract LOC_R_PCm_R
 Done tract LOC_R_PFCm_L
 Done tract LOC_R_PFCm_R
 Done tract PCG_L_PCG_R
 Done tract PCG_L_PCm_R
 Done tract PCG_L_PFCm_L
 Done tract PCG_L_PFCm_R
 Done tract PCG_R_PCm_L
 Done tract PCG_R_PCm_R
 Done tract PCG_R_PFCm_L
 Done tract PCG_R_PFCm_R
 Done tract PCm_L_PCm_R
 Done tract PCm_L_PFCm_L
 Done tract PCm_L_PFCm_R
 Done tract PCm_R_PFCm_L
 Done tract PCm_R_PFCm_R
 Done tract PFCm_L_PFCm_R

 Generated histograms in /Volumes/AndrewElements/data/nki/tract_stats/dmn/figures.


<Figure size 3600x2880 with 0 Axes>